# Template Live data

### Import Library

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import requests
import pickle
import os
from pathlib import Path

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

In [2]:
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_3_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36"
]

HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
VNDIRECT_DATA_HISTORY_URL = 'https://dchart-api.vndirect.com.vn/dchart/history'
URL = VNDIRECT_DATA_HISTORY_URL

In [3]:
def getStockHistoryData(ticker, timestamp_from=0, timestamp_to=0):
    if timestamp_from == 0:
        three_months = date.today() + relativedelta(months=-3)
        timestamp_from = datetime.strptime(three_months.strftime("%m/%d/%Y") + ', 00:00:0', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()
    if timestamp_to == 0:
        timestamp_to = datetime.strptime(date.today().strftime("%m/%d/%Y") + ', 23:59:00', "%m/%d/%Y, %H:%M:%S")\
            .timestamp()

    params = {
        "resolution": "5",
        "symbol": str(ticker),
        "from": int(timestamp_from),
        "to": int(timestamp_to)
    }

    x = requests.get(URL, params=params, headers=HEADERS)
    response = x.json()

    import numpy as np
    import pandas as pd

    timestamp = np.array(response['t']).astype(int)
    close = np.array(response['c']).astype(float)
    open = np.array(response['o']).astype(float)
    high = np.array(response['h']).astype(float)
    low = np.array(response['l']).astype(float)
    volume = np.array(response['v']).astype(int)

    dataset = pd.DataFrame({'Time': timestamp, 'Open': list(open), 'High': list(high), 'Low': list(low),
                            'Close': list(close), 'Volume': list(volume)},
                           columns=['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    return dataset

In [4]:
def prepareData(htd):
    if 'Time' in htd.columns:
        from datetime import datetime

        htd['DateStr'] = htd.apply(
            lambda x: datetime.fromtimestamp(x['Time']).strftime("%Y-%m-%d %H:%M:%S"), axis=1)

    htd['Date'] = pd.to_datetime(htd['DateStr'])
    ticker_data = htd.set_index('Date')
    ticker_data.drop(columns=['Time', 'DateStr'], inplace=True)
    return ticker_data

### Load Price Data

In [5]:
ticker = "VN30F1M"
htd = getStockHistoryData(ticker, 0, 0)
ticker_data = prepareData(htd)
ticker_data = ticker_data.dropna()
ticker_data

,Open,High,Low,Close,Volume
Date,,,,,
2024-06-12 09:00:00,1302.2,1303.1,1301.4,1303.1,5498
2024-06-12 09:05:00,1303.2,1304.1,1302.9,1304.1,3514
2024-06-12 09:10:00,1304.0,1304.2,1302.8,1303.0,2249
2024-06-12 09:15:00,1303.0,1304.1,1301.5,1301.8,4969
2024-06-12 09:20:00,1301.8,1303.3,1301.6,1303.3,3093
...,...,...,...,...,...
2024-09-12 10:50:00,1301.6,1301.8,1301.2,1301.4,2496
2024-09-12 10:55:00,1301.5,1301.7,1301.2,1301.3,1360
2024-09-12 11:00:00,1301.2,1301.2,1300.4,1300.6,2929


In [6]:
ticker_data["ma_line"] = ticker_data["Close"].rolling(20).mean()
ticker_data['above_ma'] = ticker_data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
ticker_data['below_ma'] = ticker_data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
ticker_data['total_above_ma_1w'] = ticker_data['above_ma'].rolling(250).sum()
ticker_data['total_below_ma_1w'] = ticker_data['below_ma'].rolling(250).sum()
ticker_data['trend'] = ticker_data.apply(lambda r: 'switch' if r['total_above_ma_1w'] == r['total_below_ma_1w'] else (
    'up' if r['total_above_ma_1w'] > r['total_below_ma_1w'] else 'down'), axis=1)

In [7]:
ticker_data['Close_9h'] = ticker_data['Close']

In [8]:
def bar9h(tick):
    tick=tick[(tick.index.hour == 9) & (tick.index.minute == 0)]
    if len(tick):
        return tick[0]
day_data = ticker_data.resample('D').agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'Close_9h': bar9h,
        'trend': bar9h
    })
day_data.dropna(subset=['Close'], inplace=True)

In [9]:
day_data['Pivot'] = day_data.apply(lambda row: (row['High'] + row['Low'] + row['Close']) / 3, axis=1)
day_data['Pivot'] = day_data['Pivot'].shift(1)
day_data['High_s'] = day_data['High'].shift(1)
day_data['Low_s'] = day_data['Low'].shift(1)
day_data['first_open_vs_pivot'] = day_data.apply(lambda r: 1000 * (r['Open'] - r['Pivot']) / r['Open'], axis=1)
day_data['first_close_vs_pivot'] = day_data.apply(lambda r: 1000 * (r['Close_9h'] - r['Pivot']) / r['Close_9h'], axis=1)

In [10]:
day_data

,Open,Close,High,Low,Close_9h,trend,Pivot,High_s,Low_s,first_open_vs_pivot,first_close_vs_pivot
Date,,,,,,,,,,,
2024-06-12,1302.2,1330.7,1332.0,1301.4,1303.1,down,NaN,NaN,NaN,NaN,NaN
2024-06-13,1331.0,1335.0,1338.6,1324.9,1331.0,down,1321.366667,1332.0,1301.4,7.237666,7.237666
2024-06-14,1333.5,1317.0,1335.9,1317.0,1332.1,down,1332.833333,1338.6,1324.9,0.499938,-0.550509
2024-06-17,1315.5,1308.0,1315.8,1305.3,1313.9,down,1323.300000,1335.9,1317.0,-5.929304,-7.154274
2024-06-18,1312.7,1308.5,1322.4,1308.5,1311.9,down,1309.700000,1315.8,1305.3,2.285366,1.676957
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-06,1307.6,1309.0,1311.3,1305.6,1308.0,up,1310.333333,1318.6,1304.7,-2.090344,-1.783894
2024-09-09,1299.2,1307.7,1308.9,1298.6,1302.0,up,1308.633333,1311.3,1305.6,-7.260878,-5.094726
2024-09-10,1310.8,1294.1,1313.6,1289.4,1313.1,down,1305.066667,1308.9,1298.6,4.373919,6.117838


In [11]:
def get_low_pivot_table():
    algo_path = Path(os.getcwd()).parent.parent.parent
    pivot_table_file = str(algo_path) + '/VN30ps/high_low/low_pivot_table.pickle'
    if pivot_table_file:
        with open(pivot_table_file, 'rb') as f:
            return pickle.load(f)


def get_high_pivot_table():
    algo_path = Path(os.getcwd()).parent.parent.parent
    pivot_table_file = str(algo_path) + '/VN30ps/high_low/high_pivot_table.pickle'
    if pivot_table_file:
        with open(pivot_table_file, 'rb') as f:
            return pickle.load(f)

In [12]:
def predict_low(data):
    pivot_df = get_low_pivot_table()
    if not len(pivot_df):
        return
    found_i = None
    for i in pivot_df.index:
        if i.left < data['first_close_vs_pivot'] <= i.right:
            found_i = i
            break
    found_c = None
    for c in pivot_df.columns:
        if c.left < data['first_open_vs_pivot'] <= c.right:
            found_c = c
            break
    found = pivot_df.loc[pivot_df.index == found_i, found_c]
    if len(found):
        val = found.values[0]
        return (val * data['Pivot']) / 1000 + data['Low_s']


def predict_high(data):
    pivot_df = get_high_pivot_table()
    if not len(pivot_df):
        return
    found_i = None
    for i in pivot_df.index:
        if i.left < data['first_close_vs_pivot'] <= i.right:
            found_i = i
            break
    found_c = None
    for c in pivot_df.columns:
        if c.left < data['first_open_vs_pivot'] <= c.right:
            found_c = c
            break

    found = pivot_df.loc[pivot_df.index == found_i, found_c]
    if len(found):
        val = found.values[0]
        return (val * data['Pivot']) / 1000 + data['High_s']

In [13]:
day_data.dropna(subset=['Pivot'], inplace=True)
day_data['high_predict'] = day_data.apply(lambda r: predict_high(r), axis=1)
day_data['low_predict'] = day_data.apply(lambda r: predict_low(r), axis=1)

In [14]:
last_data = day_data.iloc[-1]
last_data

Open                         1300.0
Close                        1299.5
High                         1302.1
Low                          1297.9
Close_9h                     1299.1
trend                          down
Pivot                        1292.3
High_s                       1297.3
Low_s                        1286.6
first_open_vs_pivot        5.923077
first_close_vs_pivot       5.234393
high_predict            1302.948333
low_predict             1294.318985
Name: 2024-09-12 00:00:00, dtype: object

In [15]:
day_data[['trend', 'Close_9h', 'High', 'high_predict', 'Low', 'low_predict']]

,trend,Close_9h,High,high_predict,Low,low_predict
Date,,,,,,
2024-06-13,down,1331.0,1338.6,1343.944097,1324.9,1316.279780
2024-06-14,down,1332.1,1335.9,1338.175825,1317.0,1321.390652
2024-06-17,down,1313.9,1315.8,1324.416123,1305.3,1303.783844
2024-06-18,down,1311.9,1322.4,1321.777802,1308.5,1309.887890
2024-06-19,up,1310.9,1325.0,1333.249132,1298.8,1307.851976
...,...,...,...,...,...,...
2024-09-06,up,1308.0,1311.3,1316.548813,1305.6,1295.900855
2024-09-09,up,1302.0,1308.9,1316.691458,1298.6,1297.869737
2024-09-10,down,1313.1,1313.6,1318.948039,1289.4,1308.908264


In [16]:
day_data['diff_high'] = day_data.apply(lambda r: r['high_predict'] - r['High'], axis=1)
day_data['diff_low'] = day_data.apply(lambda r: r['low_predict'] - r['Low'], axis=1)
print(day_data['diff_high'].abs().sum() / len(day_data))
print(day_data['diff_low'].abs().sum() / len(day_data))
print(day_data['diff_high'].max())
print(day_data['diff_high'].min())
print(day_data['diff_low'].max())
print(day_data['diff_low'].min())

4.773232163042213
5.162485724333024
12.457097178290041
-13.384414406945098
28.025625933595848
-12.708001696447809


In [17]:
len(day_data[abs(day_data.diff_high) > 6])

21

In [18]:
day_data[(day_data.diff_low < 0) & (day_data.trend == 'up')]

,Open,Close,High,Low,Close_9h,trend,Pivot,High_s,Low_s,first_open_vs_pivot,first_close_vs_pivot,high_predict,low_predict,diff_high,diff_low
Date,,,,,,,,,,,,,,,
2024-07-03,1297.1,1307.0,1309.5,1294.5,1296.5,up,1292.566667,1299.0,1283.5,3.494976,3.033809,1302.494203,1289.458485,-7.005797,-5.041515
2024-07-04,1307.0,1308.6,1313.3,1303.5,1306.7,up,1303.666667,1309.5,1294.5,2.550370,2.321369,1315.450264,1299.066755,2.150264,-4.433245
2024-07-05,1308.5,1316.3,1316.5,1307.2,1309.7,up,1308.466667,1313.3,1303.5,0.025474,0.941691,1314.257904,1303.798675,-2.242096,-3.401325
2024-07-29,1289.9,1287.8,1294.7,1287.8,1291.2,up,1282.266667,1285.0,1276.8,5.917771,6.918629,1299.686113,1286.640366,4.986113,-1.159634
2024-07-30,1286.1,1292.4,1292.4,1284.2,1285.6,up,1290.100000,1294.7,1287.8,-3.110178,-3.500311,1290.098423,1281.181121,-2.301577,-3.018879
2024-07-31,1292.0,1304.3,1304.3,1291.1,1292.3,up,1289.666667,1292.4,1284.2,1.805986,2.037711,1296.953490,1286.946172,-7.346510,-4.153828
2024-08-02,1266.2,1275.9,1277.5,1256.3,1264.6,up,1284.066667,1306.2,1271.0,-14.110462,-15.393537,1285.318885,1246.947875,7.818885,-9.352125
2024-08-12,1262.5,1270.0,1270.4,1258.7,1262.9,up,1259.600000,1266.2,1247.7,2.297030,2.613033,1270.537374,1250.555953,0.137374,-8.144047
2024-08-13,1269.0,1267.0,1270.1,1262.7,1268.5,up,1266.366667,1270.4,1258.7,2.075125,1.681776,1274.871223,1261.396558,4.771223,-1.303442


In [19]:
day_data[(abs(day_data.diff_high) > 6) & (day_data.trend == 'up')]

,Open,Close,High,Low,Close_9h,trend,Pivot,High_s,Low_s,first_open_vs_pivot,first_close_vs_pivot,high_predict,low_predict,diff_high,diff_low
Date,,,,,,,,,,,,,,,
2024-06-19,1309.8,1317.0,1325.0,1298.8,1310.9,up,1313.133333,1322.4,1308.5,-2.544918,-1.703664,1333.249132,1307.851976,8.249132,9.051976
2024-07-03,1297.1,1307.0,1309.5,1294.5,1296.5,up,1292.566667,1299.0,1283.5,3.494976,3.033809,1302.494203,1289.458485,-7.005797,-5.041515
2024-07-10,1318.9,1310.0,1321.9,1309.7,1319.6,up,1315.633333,1321.9,1304.5,2.476811,3.005961,1330.097465,1311.903420,8.197465,2.203420
2024-07-31,1292.0,1304.3,1304.3,1291.1,1292.3,up,1289.666667,1292.4,1284.2,1.805986,2.037711,1296.953490,1286.946172,-7.346510,-4.153828
2024-08-02,1266.2,1275.9,1277.5,1256.3,1264.6,up,1284.066667,1306.2,1271.0,-14.110462,-15.393537,1285.318885,1246.947875,7.818885,-9.352125
2024-08-15,1268.0,1267.7,1269.7,1263.4,1269.5,up,1269.900000,1273.7,1268.0,-1.498423,-0.315085,1277.387765,1268.580194,7.687765,5.180194
2024-08-16,1271.5,1288.0,1289.9,1268.2,1270.2,up,1266.933333,1269.7,1263.4,3.591559,2.571773,1276.515586,1267.735025,-13.384414,-0.464975
2024-08-20,1301.6,1302.0,1308.8,1298.3,1299.5,up,1295.633333,1300.6,1288.3,4.584102,2.975503,1301.437829,1296.401734,-7.362171,-1.898266
2024-08-21,1300.8,1312.4,1314.5,1297.2,1299.7,up,1303.033333,1308.8,1298.3,-1.716892,-2.564694,1306.760241,1289.549876,-7.739759,-7.650124


In [20]:
day_data[(abs(day_data.diff_high) > 6) & (day_data.trend == 'down')]

,Open,Close,High,Low,Close_9h,trend,Pivot,High_s,Low_s,first_open_vs_pivot,first_close_vs_pivot,high_predict,low_predict,diff_high,diff_low
Date,,,,,,,,,,,,,,,
2024-06-17,1315.5,1308.0,1315.8,1305.3,1313.9,down,1323.300000,1335.9,1317.0,-5.929304,-7.154274,1324.416123,1303.783844,8.616123,-1.516156
2024-06-21,1322.6,1320.9,1323.4,1311.7,1321.2,down,1322.733333,1327.8,1316.7,-0.100812,-1.160561,1334.319992,1320.589233,10.919992,8.889233
2024-06-25,1293.9,1287.7,1294.0,1280.1,1292.5,down,1297.833333,1317.3,1285.5,-3.039905,-4.126370,1306.457097,1273.169913,12.457097,-6.930087
2024-07-01,1277.7,1285.0,1285.8,1271.7,1277.5,down,1279.866667,1292.8,1272.5,-1.695755,-1.852577,1294.241148,1272.834469,8.441148,1.134469
2024-07-02,1284.7,1295.2,1299.0,1283.5,1286.6,down,1280.833333,1285.8,1271.7,3.009782,4.482098,1290.163959,1280.405475,-8.836041,-3.094525
2024-07-17,1309.0,1303.2,1321.8,1294.7,1308.2,down,1303.500000,1308.7,1299.1,4.201681,3.592723,1314.636648,1303.981323,-7.163352,9.281323
2024-07-18,1303.1,1297.5,1305.0,1287.9,1303.9,down,1306.566667,1321.8,1294.7,-2.660323,-2.045147,1315.061681,1287.927717,10.061681,0.027717
2024-07-22,1291.3,1300.0,1304.4,1291.3,1293.6,down,1301.200000,1311.2,1295.3,-7.666692,-5.875077,1296.367043,1286.167245,-8.032957,-5.132755
2024-07-24,1283.3,1283.5,1287.4,1273.3,1282.7,down,1289.333333,1303.7,1280.1,-4.701421,-5.171383,1295.928295,1273.029422,8.528295,-0.270578
